# Anomalies case

## Task Requirements

The sample includes information about transactions on some real estate market. The presented features are:

- transaction_price - reported transaction price of an apartment; 
- square_meters - an area of an apartment in square meters; 
- num_bedrooms - number of bedrooms in an apartment; 
- floor - on which floor an apartment is located; 

The most common way to reduce transfer tax is to under-declare the transaction price while receiving more value for a deal. 

Based on the given dataset, please investigate if there is any evidence of tax evasion.

In [95]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression

In [96]:
df = pd.read_csv('transactions_data.txt', sep=' ', header=None)
df.columns = ['transaction_price', 'square_meters', 'num_bedrooms', 'floor']
df.head()

,transaction_price,square_meters,num_bedrooms,floor
0,235072,57,1,5
1,246370,50,1,4
2,229739,50,1,5
3,235836,106,1,1
4,239066,51,1,6


In [97]:
df.shape[0]

100

## Data Preparation 

Actually we want to catch up intentional anomalies, that's why some methods like classification and clusterization models could  be not effective: it seems that price only should be different in the same cluster or class and we don't know which one is incorrect. 

So here we suppose that the transaction price is the result of linear regression which depends on square, bedrooms number and floor number.
For this purpose we're going to reshape the column with floor data. Also we don't know initially how many values of floors are used.

In [98]:
df_floor = df.copy()

u = df["floor"].unique() #put all unique values to separate array 
add_col = len(u) #define the number of new columns for adding to primary dataframe 

#add firstly columns with zero values 
for i in range (0, add_col):
    s = str(u[i])
    df_floor[s] = 0

n_col = df.shape[1] #the number of columns in primary dataframe

#fill with 1 those cells which are equal to corresponding floor columns values 
for i in range (0, df_floor.shape[0]):
    for j in range (n_col, n_col + add_col):
        if int(df_floor.columns[j]) == df_floor.iloc[i]["floor"]:
            df_floor.at[i,df_floor.columns[j]] = 1
    
df_floor.drop("floor", axis=1, inplace=True)
df_res = df_floor.copy()

print ("Dataframe for further modelling consists of " + str(df_res.shape[0]) + " rows and " + str(df_res.shape[1]) + " columns")
df_res[:3]

Dataframe for further modelling consists of 100 rows and 9 columns


,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2
0,235072,57,1,1,0,0,0,0,0
1,246370,50,1,0,1,0,0,0,0
2,229739,50,1,1,0,0,0,0,0


## Linear Regression 

In [99]:
#function results
y = np.array(df_res["transaction_price"]) 

#function attributes 
df_reg = df_res.copy()
df_reg.drop("transaction_price", axis=1, inplace=True)
x = np.array(df_reg)

#use the regression model and find out its coefficients 
model = LinearRegression().fit(x, y)
b0 = model.intercept_
bn = model.coef_

In [100]:
b0

135146.79241841848

In [101]:
bn

array([  1440.659714  ,   5957.2036893 ,  11616.12610406,   7489.26464903,
       -29777.89010295,  30728.90894864,   9550.60570014, -29607.01529892])

It seems that the higher you live the more you should pay:)

Now we're going to calculate the corresponding model result for each row of primary dataframe.

In [102]:
fin_df = df_res.copy()
fin_df["reg_sum"] = 0 #that's new column with regression's result

for i in range (0, fin_df.shape[0]):
    reg_sum = b0
    for j in range(1, len(bn)): 
        reg_sum = reg_sum + (bn[j-1] * fin_df.iloc[i][j])
    fin_df.at[i,["reg_sum"]] = int(reg_sum)

In [103]:
fin_df[:5]

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum
0,235072,57,1,1,0,0,0,0,0,234837
1,246370,50,1,0,1,0,0,0,0,220626
2,229739,50,1,1,0,0,0,0,0,224753
3,235836,106,1,0,0,1,0,0,0,264036
4,239066,51,1,0,0,0,1,0,0,245306


## Anomalies Investigation via Standart Score

Now we need to define anomalies. The more difference between transaction price and regression result the higher probability of incorrectness.

We will use z-score or standard score - basically it's the number of standard deviations by which the value is above or below the mean value of what is being measured. Raw scores above the mean have positive standard scores, while those below the mean have negative standard scores.

Here we're going to evaluate the difference between transaction price and regression result as a random variable.

The standard score is calculating by following approach: z = (x - Xavg) / S, where x is a variable value, Xavg - an average value for full sample (arithmetical mean: Xavg = sum(x) / N, where N is the observation number), S - a standart deviasion of full sample).

S is the square root of standart variance, S = sqrt ( sum ((x - Xavg)^2) / N )

In [104]:
fin_df["delta"] = fin_df["reg_sum"] - fin_df["transaction_price"] #respectively it's x in description above

avg = fin_df.mean()["delta"] #Xavg

fin_df["delta_sq"] = (fin_df["delta"] - avg) ** 2 #(x - Xavg)^2

count = fin_df.count()["delta_sq"] #N

sum = fin_df.sum()["delta_sq"] #sum ((x - Xavg)^2)

S = ( sum / (count) ) ** (1/2) #S

fin_df["Z"] = (fin_df["delta"] - avg) / S #Z

print("Average difference between regression result and transactions price: " + str(avg))
print ("Observations number: " + str(count))
print ("Standart deviation : " + str(S))

Average difference between regression result and transactions price: 1775.98
Observations number: 100
Standart deviation : 37743.7500958715


In [105]:
fin_df[:3]

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum,delta,delta_sq,Z
0,235072,57,1,1,0,0,0,0,0,234837,-235,4.044041e+06,-0.053280
1,246370,50,1,0,1,0,0,0,0,220626,-25744,7.573493e+08,-0.729127
2,229739,50,1,1,0,0,0,0,0,224753,-4986,4.572437e+07,-0.179155


Now z-score is a key for anomalies defining. 

We will use the three-sigma rule: random variables which are normally distributed are characterized by a deviation of S from the expected value with a probability of 68%, by 2S - by 95%, by 3S - by 99.7%.

Regarding 2S the corresponding values of z-score are 1.96 (the result is lower than excepted and that's target result of this task) and -1.96 (the result is higher than excepted).

Here we're going to use 1.65 score (corresponding 90%) due to intentionally of anomalies.

In [106]:
tax = pd.DataFrame(columns = fin_df.columns)
remove = pd.DataFrame(columns = fin_df.columns)
further = fin_df.copy()

j_tax = 0
j_remove = 0
j_fur = 0
for i in range (fin_df.shape[0]):
    
    if fin_df.iloc[i][-1] > 1.65:
        tax.loc[j_tax] = fin_df.loc[i]
        further.drop(i, axis = 0, inplace=True)
        j_tax = j_tax + 1 
        
    elif fin_df.iloc[i][-1] < -1.65:
        remove.loc[j_remove] = fin_df.loc[i]
        further.drop(i, axis = 0, inplace=True)
        j_remove = j_remove + 1 
        
    else:
        j_fur = j_fur + 1 
    further.index = np.arange(len(further))    
further[:3]

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum,delta,delta_sq,Z
0,235072,57,1,1,0,0,0,0,0,234837,-235,4.044041e+06,-0.053280
1,246370,50,1,0,1,0,0,0,0,220626,-25744,7.573493e+08,-0.729127
2,229739,50,1,1,0,0,0,0,0,224753,-4986,4.572437e+07,-0.179155


In [107]:
remove

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum,delta,delta_sq,Z
0,265323.0,50.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,183359.0,-81964.0,7.012384e+09,-2.218645
1,957630.0,435.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,793151.0,-164479.0,2.764072e+10,-4.404835
2,501816.0,199.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,403974.0,-97842.0,9.923742e+09,-2.639324
3,315313.0,50.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,243865.0,-71448.0,5.361751e+09,-1.940029
4,266236.0,50.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,189316.0,-76920.0,6.193057e+09,-2.085007


In [108]:
tax

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum,delta,delta_sq,Z
0,215051.0,125.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,327143.0,112092.0,1.216962e+10,2.922763
1,269469.0,155.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,334628.0,65159.0,4.017407e+09,1.679298
2,226882.0,133.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,308891.0,82009.0,6.437337e+09,2.125730


## Final 

Finally, we can make some iterations: just repeat the same evaluation and remove from further calculations cases with higher prices which are not interesting for us and cases with anomally lower prices which are target result of this task.
The separate dataframe should be consisted of such rows.

In [109]:
tax = pd.DataFrame(columns = fin_df.columns) 
remove = pd.DataFrame(columns = fin_df.columns)
further = fin_df.copy()
input_frame = fin_df.copy()
k = 1 #stopfactor
j_tax = 0
j_remove = 0
j_fur = 0
iterations = 0

while iterations < 3: 
    
    for i in range (0, input_frame.shape[0]):
        k = 0
        
        if input_frame.iloc[i][-1] > 1.65:
            tax.loc[j_tax] = input_frame.loc[i]
            further.drop(i, axis=0, inplace=True)
            j_tax = j_tax + 1 
            k = k + 1
            
        elif input_frame.iloc[i][-1] < -1.65:
            remove.loc[j_remove] = input_frame.loc[i]
            further.drop(i, axis=0, inplace=True)
            j_remove = j_remove + 1 
            k = k + 1
            
        else:
            j_fur = j_fur + 1 
            
    iterations = iterations + 1        
    print ("Algorithm iteration number: " + str(iterations))
    print ("z-score maximum value: " + str(input_frame.max()["Z"]))
    print ("z-score minimum value: " + str(input_frame.min()["Z"]))
    print ("Dataframe shape with target underpricing cases after this iteration: " + str(tax.shape[0]))
    print ("Dataframe shape with removed overpricing cases after this iteration: " + str(remove.shape[0]))
    print ("Dataframe shape after this iteration: " + str(further.shape[0]) + "\n")

    
    further.index = np.arange(len(further)) 
    input_frame = further.copy()        
    input_frame.drop("reg_sum", axis=1, inplace=True)
    input_frame.drop("delta", axis=1, inplace=True)
    input_frame.drop("delta_sq", axis=1, inplace=True)
    input_frame.drop("Z", axis=1, inplace=True)
    new_y = np.array(input_frame["transaction_price"])
    #выведем в отдельный датафрейм все признаки - мы не знаем сколько их из-за используемых этажей
    
    df_reg = input_frame.copy()
    df_reg.drop("transaction_price", axis=1, inplace=True)
    new_x = np.array(df_reg)

    #используем модель регрессии и получаем набор коэффициентов
    new_model = LinearRegression().fit(new_x, new_y)
    new_b0 = new_model.intercept_
    new_bn = new_model.coef_    
    
    #Теперь для каждой строки рассчитаем значение цены, определенное моделью регрессии
    new_fin_df = input_frame.copy()
    new_fin_df["reg_sum"] = 0 #создаем новый столбец со значением цены, рассчитанным по построенной регрессии

    for i in range (0, new_fin_df.shape[0]):
        new_reg_sum = new_b0
        for j in (1, len(new_bn)): 
#0 столбец - это transaction_price, последний - это reg_sum. Таким образом отсчет столбцов - от 1 до длинны массива с bn 
#коэффициентами, а значение в них берем по j - 1 индексу, значение столбца - по j-му
            new_reg_sum = new_reg_sum + (new_bn[j-1] * new_fin_df.iloc[i][j])
        new_fin_df.at[i,["reg_sum"]] = int(new_reg_sum)
        
        
    new_fin_df["delta"] = new_fin_df["reg_sum"] - new_fin_df["transaction_price"] #соответственно, это х из объяснения выше
    new_avg = new_fin_df.mean()["delta"] #Xavg

    new_fin_df["delta_sq"] = (new_fin_df["delta"] - new_avg) ** 2 #(x - Xavg)^2

    new_count = new_fin_df.count()["delta_sq"] #N

    new_sum = new_fin_df.sum()["delta_sq"] #sum ((x - Xavg)^2)

    new_S = ( new_sum / (new_count) ) ** (1/2) #S

    new_fin_df["Z"] = (new_fin_df["delta"] - new_avg) / S #Z
    
    #этот фрейм без выброшенных аномалий запускаем в цикл заново
    input_frame = new_fin_df.copy() 


Algorithm iteration number: 1
z-score maximum value: 2.9227625691615264
z-score minimum value: -4.404834696544511
Dataframe shape with target underpricing cases after this iteration: 3
Dataframe shape with removed overpricing cases after this iteration: 5
Dataframe shape after this iteration: 92

Algorithm iteration number: 2
z-score maximum value: 1.8487408608375344
z-score minimum value: -2.016561331695737
Dataframe shape with target underpricing cases after this iteration: 5
Dataframe shape with removed overpricing cases after this iteration: 7
Dataframe shape after this iteration: 88

Algorithm iteration number: 3
z-score maximum value: 1.595522734510249
z-score minimum value: -1.5565487925056456
Dataframe shape with target underpricing cases after this iteration: 5
Dataframe shape with removed overpricing cases after this iteration: 7
Dataframe shape after this iteration: 88



## Result

In [110]:
tax

,transaction_price,square_meters,num_bedrooms,5,4,1,6,3,2,reg_sum,delta,delta_sq,Z
0,215051.0,125.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,327143.0,112092.0,1.216962e+10,2.922763
1,269469.0,155.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,334628.0,65159.0,4.017407e+09,1.679298
2,226882.0,133.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,308891.0,82009.0,6.437337e+09,2.125730
3,179527.0,75.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,242983.0,63456.0,4.869027e+09,1.848741
4,155162.0,50.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,211286.0,56124.0,3.899555e+09,1.654484
